In [1]:
# ! pip install pyrtools
# ! pip install opencv-python
# pip install Pillow
# ! pip install scipy

In [2]:
import cv2
import numpy as np
import pyrtools as pt
from PIL import Image
from scipy import signal

## COLOR CLUTTER

In [3]:
def conv2(x, y, mode=None):
    
    if mode == 'same':
        return np.rot90(signal.convolve2d(np.rot90(x, 2), np.rot90(y, 2), mode=mode), 2)
    else:
        return signal.convolve2d(x, y)
        

def RRoverlapconv(kernel, in_):
    out = conv2(in_, kernel, mode='same')
    rect = np.ones_like(in_)

    overlapsum = conv2(rect, kernel, 'same')
    out = np.sum(kernel) * out / overlapsum
    return out



def RRgaussfilter1D(halfsupport, sigma, center=0):
    t = list(range(-halfsupport, halfsupport+1))
    kernel = [np.exp(-(x-center) **2 /(2* sigma ** 2)) for x in t]
    kernel = kernel/sum(kernel)
    
    return kernel[:,None]
    

def computeColorClutter(L_pyr, a_pyr, b_pyr, sigD):
    
    numlevels = len(L_pyr);

    
    if len(a_pyr)!=numlevels or len(b_pyr)!=numlevels:
        print('L, a, and b channels must have the same number of levels in the Gaussian pyramid\n')

    covMx = {}
    clutter_levels = [0] * numlevels
    DL = [0] * numlevels
    Da = [0] * numlevels
    Db = [0] * numlevels
    
    
    deltaL2 = 0.0007 ** 2
    deltaa2 = 0.1 ** 2
    deltab2 = 0.05 ** 2
    
    bigG = RRgaussfilter1D(round(2*sigD), sigD)
    
    for i in range(0, numlevels):
        DL[i] = RRoverlapconv(bigG, L_pyr[(i,0)])
        DL[i] = RRoverlapconv(bigG.T, DL[i])   # E(L)
        Da[i] = RRoverlapconv(bigG, a_pyr[(i,0)])
        Da[i] = RRoverlapconv(bigG.T, Da[i])   # E(a)
        Db[i] = RRoverlapconv(bigG, b_pyr[(i,0)]);
        Db[i] = RRoverlapconv(bigG.T, Db[i])    # E(b)

    
        # dict idea
        covMx[(i,0,0)] = RRoverlapconv(bigG, L_pyr[(i,0)] ** 2)
        covMx[(i,0,0)] = RRoverlapconv(bigG.T, covMx[(i,0,0)]) - DL[i] ** 2 + deltaL2  # cov(L,L) + deltaL2
        covMx[(i,0,1)] = RRoverlapconv(bigG, L_pyr[(i,0)] * a_pyr[(i,0)])
        covMx[(i,0,1)] = RRoverlapconv(bigG.T, covMx[(i,0,1)]) - DL[i] * Da[i]        # cov(L,a)
        covMx[(i,0,2)] = RRoverlapconv(bigG, L_pyr[(i,0)] * b_pyr[(i,0)])
        covMx[(i,0,2)] = RRoverlapconv(bigG.T, covMx[(i,0,2)]) - DL[i] * Db[i]        # cov(L,b)
        covMx[(i,1,1)] = RRoverlapconv(bigG, a_pyr[(i,0)] ** 2)
        covMx[(i,1,1)] = RRoverlapconv(bigG.T, covMx[(i,1,1)]) - Da[i] ** 2 + deltaa2  # cov(a,a) + deltaa2
        covMx[(i,1,2)] = RRoverlapconv(bigG, a_pyr[(i,0)] * b_pyr[(i,0)])
        covMx[(i,1,2)] = RRoverlapconv(bigG.T, covMx[(i,1,2)]) - Da[i] * Db[i]        # cov(a,b)
        covMx[(i,2,2)] = RRoverlapconv(bigG, b_pyr[(i,0)] ** 2)    
        covMx[(i,2,2)] = RRoverlapconv(bigG.T, covMx[(i,2,2)]) - Db[i] ** 2 + deltab2;  # cov(b,b) + deltab2

    

        
        detIm = covMx[(i,0,0)]*(covMx[(i,1,1)]*covMx[(i,2,2)]-covMx[(i,1,2)]*covMx[(i,1,2)])\
        - covMx[(i,0,1)]*(covMx[(i,0,1)]*covMx[(i,2,2)]-covMx[(i,1,2)]*covMx[(i,0,2)])\
        + covMx[(i,0,2)]*(covMx[(i,0,1)]*covMx[(i,1,2)]-covMx[(i,1,1)]*covMx[(i,0,2)])

        
        clutter_levels[i] = np.sqrt(detIm) ** (1/3)
    return clutter_levels

In [4]:
def colorClutter(inputImage, numlevels, pool_sigma=3, pix=1):
    
    if isinstance(inputImage, list):
        L_pyr = inputImage[0]
        a_pyr = inputImage[1]
        b_pyr = inputImage[2]
        
    else:
        pass #TODO
        return 0
    
    clutter_levels = computeColorClutter(L_pyr, a_pyr, b_pyr, pool_sigma);
    
    kernel_1d = np.array([[0.05, 0.25, 0.4, 0.25, 0.05]])
    kernel_2d = conv2(kernel_1d, kernel_1d.T)
    
    clutter_map = clutter_levels[0]
    for scale in range(1,numlevels):
        clutter_here = clutter_levels[scale]
        
        for kk in range(scale, 0, -1):
            # TODO
#             start= np.array([[0,0]])
#             step = np.array([[2,2]])
#             stop = step * (np.floor( (start - np.ones_like(start))/step) + len(clutter_here))
            clutter_here = pt.upConv(image=clutter_here, filt=kernel_2d, edge_type='reflect1', step=[2,2], start=[0,0])
    
        common_sz = min(clutter_map.shape[0], clutter_here.shape[0]), min(clutter_map.shape[1], clutter_here.shape[1])
        for i in range(0, common_sz[0]):
            for j in range(0, common_sz[1]):
                 clutter_map[i][j] = max(clutter_map[i][j], clutter_here[i][j])

    
    return clutter_levels, clutter_map

## CONTRAST CLUTTER

In [5]:
# Test is passed: Diffrent test cases manually checked by MATLAB
def DoG1filter(a,sigma):
    
    sigi = 0.71 * sigma
    sigo = 1.14 * sigma
    
    t = range(-a, a+1)
    
    gi = [np.exp(-x ** 2 /(2 * sigi ** 2)) for x in t]
    gi = gi/sum(gi)
    go = [np.exp(- x ** 2/(2 * sigo ** 2)) for x in t]
    go = go/sum(go)
    return gi,go

In [6]:
# Test is passed: Diffrent test cases manually checked by MATLAB
def addborder(im,xbdr,ybdr,arg):
    """
    imnew = addborder(im,xborder,yborder,arg)  Make image w/added border.
    imnew = addborder(im,5,5,128)  Add 5 wide border of val 128.
    imnew = addborder (im,5,5,'even')  Even reflection.
    imnew = addborder (im,5,5,'odd')  Odd reflection.
    imnew = addborder (im,5,5,'wrap')  Wraparound.
    """
    ysize, xsize = im.shape
    
    
#     check thickness
    if (xbdr > xsize-1) or (ybdr > ysize-1):
        raise ValueError('borders must be thinner than image')
    
#     if arg is a number, fill border with its value.
    if isinstance(arg, (int, float)):
        imbig = cv2.copyMakeBorder(im, ybdr,ybdr,xbdr,xbdr, cv2.BORDER_CONSTANT, value=arg)
    
#     Even reflections
    elif arg == 'even':
        imbig = cv2.copyMakeBorder(im, ybdr,ybdr,xbdr,xbdr, cv2.BORDER_REFLECT)
        
#     Odd reflections
    elif arg == 'odd':
        imbig = cv2.copyMakeBorder(im, ybdr,ybdr,xbdr,xbdr, cv2.BORDER_REFLECT_101)
        
#    Wraparound
    elif arg == 'wrap':
        imbig = cv2.copyMakeBorder(im, ybdr,ybdr,xbdr,xbdr, cv2.BORDER_WRAP)
    else:
        raise ValueError('unknown border style')
    return imbig

In [7]:
def filt2(kernel, im1, reflect_style='odd'):
    """
    im2 = filt2(kernel,im1,reflect_style)
    Improved version of filter2 in MATLAB, which includes reflection.
    Default style is 'odd'. Also can be 'even', or 'wrap'.
    im2 = filt2(kern,image)  apply kernel with odd reflection (default).
    im2 = filt2(kern,image,'even')  Use even reflection.
    im2 = filt2(kern,image,128)  Fill with 128's.

    Ruth Rosenholtz
    """
    
    ky,kx = kernel.shape
    iy,ix = im1.shape

    # TODO: index should be checked, maybe it needs to be changed 1 pixel (getting back -1)
    imbig = addborder(im1, kx, ky, reflect_style)
    imbig = conv2(imbig, kernel, 'same')
    im2 = imbig[ky:ky+iy, kx:kx+ix]


    return im2

In [8]:
def RRcontrast1channel(pyr, DoG_sigma=2):
    
    levels = len(pyr)
    contrast = [0] * levels
    
    innerG1, outerG1 = DoG1filter(round(DoG_sigma*3), DoG_sigma)

    innerG1 = innerG1[:,None]
    outerG1 = outerG1[:,None]

    for i in range(0,levels):
        inner = filt2(innerG1, pyr[(i,0)])
        inner = filt2(innerG1.T, inner)
        outer = filt2(outerG1, pyr[(i,0)])
        outer = filt2(outerG1.T, outer)
        tmp = inner - outer
        contrast[i] = abs(tmp)
 
    return contrast

In [9]:
def contrastClutter(inputImage, numlevels, filt_sigma, pool_sigma=None, pix=1):
    
    if pool_sigma is None:
        pool_sigma = 3 * filt_sigma

    if isinstance(inputImage, list):
        L_pyr = inputImage[0]
        
    else:
        if isinstance(inputImage, str):
            im = cv2.imread(inputImage)
            if im is None:
                print('Unable to open %s image file.') #TODO: add logger
                return 0

        elif isinstance(inputImage,np.ndarray):
            im = inputImage
             
        m, n, d = im.shape
        if d == 3:
            Lab = cv2.cvtColor(im, cv2.COLOR_RGB2LAB)
            L = Lab[:,:,0]
        else: 
            # TODO: should be test
            L = im

  
        pyr = pt.pyramids.GaussianPyramid(L, height=numlevels)
        L_pyr = pyr.pyr_coeffs
        
        # TODO
    contrast = RRcontrast1channel(L_pyr, filt_sigma)
    
    m, n = len(contrast), 1
    clutter_levels = [0] * m
    bigG = RRgaussfilter1D(round(pool_sigma*2), pool_sigma)
    
    
    
    for scale in range(0,m):
        for channel in range(0,n):
    #         var(X) = E(X.^2) - E(X).^2
    #         get E(X) by filtering X with a 1-D Gaussian window separably in x and y directions
            meanD = RRoverlapconv(bigG, contrast[scale])
            meanD = RRoverlapconv(bigG.T, meanD)
    #         get E(X.^2) by filtering X.^2 with a 1-D Gaussian window separably in x and y directions
            meanD2 = RRoverlapconv(bigG, contrast[scale] ** 2)
            meanD2 = RRoverlapconv(bigG.T, meanD2)

    #         get variance by var(X) = E(X.^2) - E(X).^2
            stddevD = np.sqrt(abs(meanD2 - meanD ** 2))
            clutter_levels[scale] = stddevD

            
            
        
    kernel_1d = np.array([[0.05, 0.25, 0.4, 0.25, 0.05]])
    kernel_2d = conv2(kernel_1d, kernel_1d.T)
    
    clutter_map = clutter_levels[0]
    for scale in range(1,m):
        clutter_here = clutter_levels[scale]
        
        for kk in range(scale, 0, -1):
            # TODO
#             start= np.array([[0,0]])
#             step = np.array([[2,2]])
#             stop = step * (np.floor( (start - np.ones_like(start))/step) + len(clutter_here))
            clutter_here = pt.upConv(image=clutter_here, filt=kernel_2d, edge_type='reflect1', step=[2,2], start=[0,0])
    
        common_sz = min(clutter_map.shape[0], clutter_here.shape[0]), min(clutter_map.shape[1], clutter_here.shape[1])
        for i in range(0, common_sz[0]):
            for j in range(0, common_sz[1]):
                 clutter_map[i][j] = max(clutter_map[i][j], clutter_here[i][j])

    return clutter_levels, clutter_map

## ORIENTATION CLUTTER

In [41]:
def reduce(image0, kernel=None):
    """
    Reduce: for building Gaussian or Laplacian pyramids. 1-D separable kernels.

    imnew = reduce(im0) Reduce w/default kernel: [.05 .25 .4 .25 .05]
    imnew = reduce(im0, kern) Reduce with kern; sums to unity.

    Ruth Rosenholtz 
    """

    
    if kernel is None:
    #     Default kernel 
        kernel = [0.05, 0.25, 0.4, 0.25, 0.05]


    ysize, xsize = image0.shape

    image0 = filt2(kernel,image0) # Filter horizontally. 
#     filt2 is filter2 with reflection.
    image1 = image0[:,range(0,xsize,2)]

    image1 = filt2(kernel.T,image1) # Filter vertically.
    image2 = image1[range(0,ysize,2),:]

    return image2

In [42]:
def RRoverlapconvexpand(in_, kernel=None):
    """
    out = RRoverlapconvexpand(in_)  return an image expanded to double size,
    out = RRoverlapconvexpand(in, kernel); specify 1-D kernel with unity sum.
    """
    
    if kernel is None:
    #     Default kernel 
        kernel = [0.05, 0.25, 0.4, 0.25, 0.05]

    ysize, xsize = in_.shape
    kernel = kernel * 2 # kernel sum=2 to account for padding.

    tmp = np.zeros([ysize,2*xsize]) # First double the width
    k = list(range(0, xsize))
    k_2 = [x*2 for x in k]
    tmp[:,k_2] = in_[:,k]
    tmp = RRoverlapconv(kernel,tmp) # ..and filter horizontally. 

    out = np.zeros([2*ysize,2*xsize]) # Next double the height
    k = list(range(0, ysize))
    k_2 = [x*2 for x in k]
    out[k_2,:] = tmp[k,:]
    out = RRoverlapconv(kernel.T,out) # ..and filter vertically.

    return out

In [58]:
def HV(in_):
    out = in_[0] - in_[1]
    return out


def DD(in_):
    out = in_[3] - in_[2]
    return out


def sumorients(in_):
    out =  in_[0] + in_[1] + in_[2] + in_[3]
    return out

In [59]:
def poolnew(in_, sigma=None):
    """
    Pools with a gaussian.  Note assumes that input image is actually
    4 equal-size images, side by side.

    Usage: out = poolnew(input_image, sigma)

    """

    in1 = in_[0] #H -> first quarter
    in2 = in_[1] #V -> second quarter
    in3 = in_[2] #L -> third quarter
    in4 = in_[3] #R -> last quarter

    
    if sigma is None:
        out1 = reduce(RRoverlapconvexpand(in1))
        out2 = reduce(RRoverlapconvexpand(in2))
        out3 = reduce(RRoverlapconvexpand(in3))
        out4 = reduce(RRoverlapconvexpand(in4))
    else:
        kernel = RRgaussfilter1D(round(2*sigma), sigma);
        out1 = reduce(RRoverlapconvexpand(in1, kernel), kernel);
        out2 = reduce(RRoverlapconvexpand(in2, kernel), kernel);
        out3 = reduce(RRoverlapconvexpand(in3, kernel), kernel);
        out4 = reduce(RRoverlapconvexpand(in4, kernel), kernel);    

    out = out1, out2, out3, out4

    return out


In [60]:
def imrotate(im, angle, method, bbox):
    func_method = {'nearest':0,'bilinear':2,'bicubic':3,'cubic':3}
    func_bbox = {'loose':True,'crop':False}
    PIL_im = Image.fromarray(im)
    
    im_rot = PIL_im.rotate(angle, expand = func_bbox[bbox], resample = func_method[method])
    return np.array(im_rot)

In [61]:
def orient_filtnew(pyr, sigma=16/14):
    """
    ORIENT_FILTNEW Filters "pyr" (in principle, one level of the
        Gaussian pyramid generated by gausspyr) with 2nd
        derivative filters in 4 directions

    Usage: [hvdd] = orient_filt(pyr)
        Where hvdd is the 4 output images concatenated 
        together, in the order horizontal, vertical, up-left,
        and down-right.
    """
    

    halfsupport = round(3*sigma)   
#     halfsupport was 10, for default sigma.  We need a halfsupport of about
#     2*sigma for a single Gaussian.  Here we have three, one at -sigma, one at
#     sigma, so we should need a halfsupport of about 3*sigma.


    sigy = sigma
    sigx = sigma # Was sigx = 3*sigma.

    gx = RRgaussfilter1D(halfsupport, sigx)
    gy = RRgaussfilter1D(halfsupport, sigy, sigma)
    Ga = conv2(gx, gy.T)
    Ga = Ga/sum(sum(Ga))
    gy = RRgaussfilter1D(halfsupport, sigy)
    Gb = conv2(gx, gy.T)
    Gb = Gb/sum(sum(Gb))
    gy = RRgaussfilter1D(halfsupport, sigy, -sigma)
    Gc = conv2(gx, gy.T)
    Gc = Gc/sum(sum(Gc))
    H = -Ga+2*Gb-Gc
    V = H.T

    GGa = imrotate(Ga, 45, 'bicubic', 'crop')
    GGa = GGa/sum(sum(GGa))
    GGb = imrotate(Gb, 45, 'bicubic', 'crop')
    GGb = GGb/sum(sum(GGb))
    GGc = imrotate(Gc, 45, 'bicubic', 'crop')
    GGc = GGc/sum(sum(GGc))
    R = -GGa+2*GGb-GGc
    GGa = imrotate(Ga, -45, 'bicubic', 'crop')
    GGa = GGa/sum(sum(GGa))
    GGb = imrotate(Gb, -45, 'bicubic', 'crop')
    GGb = GGb/sum(sum(GGb))
    GGc = imrotate(Gc, -45, 'bicubic', 'crop')
    GGc = GGc/sum(sum(GGc))
    L = -GGa+2*GGb-GGc

    hout = filt2(H,pyr)
    vout = filt2(V,pyr)
    lout = filt2(L,pyr)
    rout = filt2(R,pyr)

    hvdd = hout, vout, lout, rout

    return hvdd 

In [124]:
def RROrientationOppEnergy(L_pyr, numlevels):
    """
    OPP_ENERGY    This runs the oriented opponent energy calculation that
    serves as the first stages in Bergen & Landy's (1990)
    texture segmentor, except it uses DOOG filters (which actually
    don't work as well, but at least we can more easily control the
    scale).
    """
    
    hvdd = [0] * numlevels
    hv = [0] * numlevels
    dd = [0] * numlevels
    out = [0] * numlevels
    total = [0] * numlevels
    
    noise = 1.0    # Was 1.5
    filterScale = 16/14*1.75
    poolScale = 1.75
#     These probably seem like arbitrary numbers, but it's just trying to get
#     three very different feature extraction methods to operate at basically
#     the same scales.


    for scale in range(0, numlevels):
#         Check this is the right order for Landy/Bergen. RRR
        hvdd[scale] = orient_filtnew(L_pyr[(scale,0)],filterScale) 
#         filt with 4 oriented filters 0, 45, 90, 135.  Was sigma = 16/14, orient_filtnew,
#         then 16/14*1.75 to match contrast and other scales.
#         Eventually make this sigma a variable that's passed to this routine.
#         hvdd[scale] is the 4 output images concatenated together, 
#         in the order horizontal, vertical, up-left, and down-right.

        hvdd[scale] = [x ** 2 for x in hvdd[scale]]    #local energy
        hvdd[scale] = poolnew(hvdd[scale], poolScale) #Pools with a gaussian filter.  Was effectively sigma=1, then 1.75 to match 1.75 above.
#         RRR Should look at these results and see if this is the right amount of
#         pooling for the new filters.  It was right for the Landy-Bergen
#         filters.
        hv[scale] = HV(hvdd[scale]) # get the difference image between horizontal and vertical: H-V (0-90)
        dd[scale] = DD(hvdd[scale]) # get the difference image between right and left: R-L (45-135)
#         Normalize by the total response at this scale, assuming the total
#         response is high enough.  If it's too low, we'll never see this
#         orientation.  I'm not sure what to do here -- set it to zeros and
#         it's like that's the orientation.  Maybe output the total response
#         and decide what to do later.  RRR
        total[scale] = sumorients(hvdd[scale]) + noise # add noise based upon sumorients at visibility threshold
        hv[scale] = hv[scale]/total[scale] # normalize the hv and dd image
        dd[scale] = dd[scale]/total[scale]
        out[scale] = hv[scale], dd[scale] # out is the 2 output images concatenated together, in the order of hv, dd


    return out

In [125]:
def computeOrientationClutter(L_pyr):
    """
    computes the orientation clutter maps. Returns:
    clutter_levels, a cell structure, containing the orientation clutter at a 
    number of scales specified by numlevels;  -- cell(numlevels,1) --, the n'th 
    level of which can be accessed using clutter_levels{n}{1}
 
    input:
        L_pyr
        the Gaussian pyramid of L (from CIELab color space)
        the Gaussian pyramid is computed by alternately blurring and subsampling the L channels
        
    Orientation clutter is computed as the "volume" of an orientation distribution
    ellipsoid, which is the determinant of covariance matrix. Treats cos(2 theta)
    and sin(2 theta) (computed from OrientedOppEnergy) as a two-vector, and gets
    The covariance of this two-vector.  The covariance 
    matrix can be computed efficiently through linear filtering. More 
    specifically, cov(X,Y) = E(XY)-E(X)E(Y), where E (expectation value) 
    can be approximated by filtering with a Gaussian window. 
    poolScale is set to 7/2.

    Reference (though there is no orientation clutter in this reference):
    Ruth Rosenholtz, Yuanzhen Li, Jonathan Mansfield, and Zhenlan Jin. 
    Feature Congestion: A Measure of Display Clutter. CHI '05: Proc. of the SIGCHI conference 
    on Human factors in computing systems. May 2005. 761-770.  

    Based upon RRcomputeOrientationSaliency
    Ruth Rosenholtz, May 2006

    This currently seems far too dependent on luminance contrast.  Check into
    why this is so -- I thought we were normalizing by local contrast.
    
    """

    noise = 0.001  # Was eps, but that gave too much orientation noise in the saliency maps.  Then changed to 0.000001
    poolScale = 7/2

    numlevels = len(L_pyr);
    Dc = [0] * numlevels  # mean "cos 2 theta" at distractor scale
    Ds = [0] * numlevels  # mean "sin 2 theta" at distractor scale

#     Get approximations to cos(2theta) and sin(2theta) from oriented opponent
#     energy, at each of the numlevels of the pyramid
    angles = RROrientationOppEnergy(L_pyr, numlevels)

#     Compute the two-vector [meancos, meansin] at each scale, as well as the
#     things we need to compute the mean and covariance of this two-vector at
#     the larger, distractor scale.

    bigG = RRgaussfilter1D(round(8*poolScale), 4*poolScale)
    maxbigG = max(bigG) ** 2

    
    covMx = {}
    clutter_levels = [0] * numlevels

    for i in range(0,numlevels):
        cmx = angles[i][0]
        smx = angles[i][1]

#         Pool to get means at distractor scale. In pooling, don't pool over the target
#         region (implement this by pooling with a big Gaussian, then
#         subtracting the pooling over the target region computed above.  Note,
#         however, that we first need to scale the target region pooling so
#         that its peak is the same height as this much broader Gaussian used
#         to pool over the distractor region.
                  
        Dc[i] = RRoverlapconv(bigG, cmx)
        Dc[i] = RRoverlapconv(bigG.T, Dc[i])
        Ds[i] = RRoverlapconv(bigG, smx)
        Ds[i] = RRoverlapconv(bigG.T, Ds[i])

#         Covariance matrix elements.  Compare with computations in
#         RRStatisticalSaliency.  I tried to match computeColorClutter, but I
#         don't remember the meaning of some of the terms I removed.  XXX
        covMx[(i,0,0)] = RRoverlapconv(bigG, cmx ** 2)
        covMx[(i,0,0)] = RRoverlapconv(bigG.T, covMx[(i,0,0)]) - Dc[i] ** 2 + noise
        covMx[(i,0,1)] = RRoverlapconv(bigG, cmx * smx)
        covMx[(i,0,1)] = RRoverlapconv(bigG.T, covMx[(i,0,1)]) - Dc[i] * Ds[i]
        covMx[(i,1,1)] = RRoverlapconv(bigG, smx ** 2)
        covMx[(i,1,1)] = RRoverlapconv(bigG.T, covMx[(i,1,1)]) - Ds[i] ** 2 + noise

#         Get determinant of covariance matrix, which is the volume of the
#         covariance ellipse
        detIm = covMx[(i,0,0)] * covMx[(i,1,1)] - covMx[(i,0,1)] ** 2
#         Take the square root considering variance is squared, and the square
#         root again, since this is the area and the contrast measure is a "length"
        clutter_levels[i] = detIm ** (1/4)

        
    return clutter_levels

In [126]:
def orientationClutter(inputImage, numlevels, pix=1):
    
    pool_sigma = 7/2


    if isinstance(inputImage, list):
        L_pyr = inputImage[0]
        
    else:
        if isinstance(inputImage, str):
            im = cv2.imread(inputImage)
            if im is None:
                raise ValueError('Unable to open %s image file.')

        elif isinstance(inputImage, np.ndarray):
            im = inputImage
             
        m, n, d = im.shape
        if d == 3:
            Lab = cv2.cvtColor(im, cv2.COLOR_RGB2LAB)
            L = Lab[:,:,0]
        else:
            raise ValueError('should be run on RGB color images.  Input image appears to be grayscale.\n')

    
#         Get Gaussian pyramid for the luminance channel
        pyr = pt.pyramids.GaussianPyramid(Lab[:,:,0], height=numlevels)
        L_pyr = pyr.pyr_coeffs

#     Compute clutter
    clutter_levels = computeOrientationClutter(L_pyr)

    
    kernel_1d = np.array([[0.05, 0.25, 0.4, 0.25, 0.05]])
    kernel_2d = conv2(kernel_1d, kernel_1d.T)
    
    clutter_map = clutter_levels[0]
    for scale in range(1,numlevels):
        clutter_here = clutter_levels[scale]
        
        for kk in range(scale, 0, -1):
            clutter_here = pt.upConv(image=clutter_here, filt=kernel_2d, edge_type='reflect1', step=[2,2], start=[0,0])
    
        common_sz = min(clutter_map.shape[0], clutter_here.shape[0]), min(clutter_map.shape[1], clutter_here.shape[1])
        for i in range(0, common_sz[0]):
            for j in range(0, common_sz[1]):
                 clutter_map[i][j] = max(clutter_map[i][j], clutter_here[i][j])

    
    
    return clutter_levels, clutter_map

## CLUTTER

In [127]:
def computeClutter(inputImage, numlevels=3, contrast_filt_sigma=1, contrast_pool_sigma=None, color_pool_sigma=3, contrast_pix=0, color_pix=0, orient_pix=0):
    
    if contrast_pool_sigma is None:
        contrast_pool_sigma = 3 * contrast_filt_sigma

    orient_pool_sigma = 7/2

    if isinstance(inputImage, str):
        im = cv2.imread(inputImage)
        if im is None:
            print('Unable to open %s image file.') #TODO: add logger
            return 0
        else:
            m, n, d = im.shape
            if d == 3:
                Lab = cv2.cvtColor(im, cv2.COLOR_RGB2LAB)
                RRLab = [0, 0, 0]
            else:
                print('should be run on RGB color images.  Input image appears to be grayscale.\n')
                return 0
  
    else:
        pass #TODO 

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,0], height=numlevels)
    RRLab[0] = pyr.pyr_coeffs

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,1], height=numlevels)
    RRLab[1] = pyr.pyr_coeffs

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,0], height=numlevels)
    RRLab[2] = pyr.pyr_coeffs
        
    
#     # compute the color clutter
    color_clutter_levels, color_clutter_map = colorClutter(RRLab, numlevels, color_pool_sigma, color_pix)
#     # compute the contrast clutter
    contrast_clutter_levels, contrast_clutter_map = contrastClutter(RRLab, numlevels, contrast_filt_sigma, contrast_pool_sigma, contrast_pix)
#     #compute the orientation clutter
    orient_clutter_levels, orientation_clutter_map = orientationClutter(RRLab, numlevels, orient_pix)

#     # output them in cell structures
    color_clutter = [color_clutter_levels, color_clutter_map]
    contrast_clutter = [contrast_clutter_levels, contrast_clutter_map]
    orientation_clutter = [orient_clutter_levels, orientation_clutter_map]


    return color_clutter, contrast_clutter, orientation_clutter

In [137]:
def getClutter_FC(filename, p=1):
    color_clutter, contrast_clutter, orient_clutter = computeClutter(filename, 3, 1, 3, 3, 0, 0, 0)
    clutter_map_fc = color_clutter[1] / 0.2088 + contrast_clutter[1] / 0.0660 + orient_clutter[1] / 0.0269
    clutter_scalar_fc = np.mean(clutter_map_fc ** p) ** (1 / p) #element wise
    return clutter_scalar_fc, clutter_map_fc


In [130]:
# test

color_clutter, contrast_clutter, orientation_clutter = computeClutter('test.jpg', 3, 1, 3, 3, 0, 0, 0)
clutter_scalar_fc, clutter_map_fc = getClutter_FC('test.jpg')

img1 = Image.fromarray(color_clutter[1]/0.01) #0.01 for test: more brightness/
img2 = Image.fromarray(contrast_clutter[1]/0.01) #0.01 for test: more brightness
img3 = Image.fromarray(orientation_clutter[1]/0.01) #0.01 for test: more brightness

img4 = Image.fromarray(clutter_map_fc) #0.01 for test: more brightness

img1.show()
img2.show()
img3.show()
img4.show()